In [1]:
!pip install sentence-splitter
!pip install SentencePiece
!pip install open_clip_torch
!pip install gradio
!pip install torch
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 1.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 549.1/549.1 kB 44.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 22.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 48.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
#------------------------------------------------- Imports -----------------------------------------
import gradio as gr
import torch
import open_clip
from transformers import AutoProcessor, AutoTokenizer, AutoImageProcessor, AutoModelForCausalLM, BlipForConditionalGeneration, Blip2ForConditionalGeneration, VisionEncoderDecoderModel
from huggingface_hub import hf_hub_download
from transformers import PegasusForConditionalGeneration, PegasusTokenizer


In [3]:
#------------------------------------ Paraphrasing model -----------------------------------------

# importing the PEGASUS Transformer model
model_name = 'tuner007/pegasus_paraphrase'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
pegasus_model = PegasusForConditionalGeneration.from_pretrained(model_name).to(torch_device)

# setting up the model
def get_response(input_text):
  first_recommendation=input_text;
  batch = tokenizer.prepare_seq2seq_batch([input_text],truncation=True,padding='longest',max_length=60, return_tensors="pt").to(torch_device)
  translated = pegasus_model.generate(**batch,max_length=60,num_beams=10, num_return_sequences=4, temperature=1.5)
  tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
  tgt_text.insert(0,first_recommendation)
  return tgt_text

In [4]:
#------------------------------------ Image Captioning model -----------------------------------------


blip_processor_base = AutoProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
blip_model_base = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

# Selecting compute unit type
device = "cuda" if torch.cuda.is_available() else "cpu"


blip_model_base.to(device)


def generate_caption(processor, model, image, tokenizer=None, use_float_16=False):
    inputs = processor(images=image, return_tensors="pt").to(device)

    if use_float_16:
        inputs = inputs.to(torch.float16)
    
    generated_ids = model.generate(pixel_values=inputs.pixel_values, max_length=50)

    if tokenizer is not None:
        generated_caption = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    else:
        generated_caption = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
   
    return generated_caption




def generate_captions(image):
    
    caption_blip_base = generate_caption(blip_processor_base, blip_model_base, image)

    return caption_blip_base


In [7]:
#------------------------------------------------ UI module ---------------------------------

# Output components for caption generator and paraphraser   
outputs_caption = [gr.outputs.Textbox(label="Caption generated by BLIP - base")] 
outputs_paraphraser = [gr.outputs.Textbox(label="Caption 1:"),gr.outputs.Textbox(label="Caption 2:"),gr.outputs.Textbox(label="Caption 3:"),gr.outputs.Textbox(label="Caption 4:"),gr.outputs.Textbox(label="Caption 5:")] 

title = "Image Caption Generator"
description = "Drag and drop images, generate captions on the fly!"

Caption_generator = gr.Interface(fn=generate_captions, 
                         inputs=gr.inputs.Image(type="pil"),
                         outputs=outputs_caption,
                         title=title,
                         description=description,
                         enable_queue=True)
Paraphraser = gr.Interface(fn=get_response,
                          title=title,
                          description=description, 
                          inputs=[gr.outputs.Textbox(label="Caption generated by BLIP - base")],
                          outputs=outputs_paraphraser)

# Launching Caption generator and paraphraser interface in series
gr.Series(Caption_generator, Paraphraser).launch(share=True)

/usr/local/lib/python3.10/dist-packages/gradio/outputs.py:22: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gradio/inputs.py:257: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gradio/deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
/usr/local/lib/python3.10/dist-packages/gradio/deprecation.py:40: UserWarning: `enable_queue` is deprecated in `Interface()`, please use it within `launch()` instead.
  warnings.warn(value)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://4fc4b0d0a7e2e87d86.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces
